In [ ]:
import os
from glob import glob
import matplotlib.pyplot as plt
import random
import cv2
import pandas as pd
import numpy as np
import matplotlib.gridspec as gridspec
import seaborn as sns
import zlib
import itertools
import sklearn
import itertools
import scipy
import skimage
from skimage.transform import resize
import csv
from tqdm import tqdm
from sklearn import model_selection
from sklearn.model_selection import train_test_split, learning_curve,KFold,cross_val_score,StratifiedKFold
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix
import keras 
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Lambda, MaxPool2D, BatchNormalization
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import models, layers, optimizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.utils import class_weight
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta, RMSprop
from keras.models import Sequential, model_from_json
from keras.layers import Activation,Dense, Dropout, Flatten, Conv2D, MaxPool2D,MaxPooling2D,AveragePooling2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.applications.mobilenet import MobileNet
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN, SVMSMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn import preprocessing
from keras.applications.nasnet import NASNetLarge, NASNetMobile
from keras.applications.mobilenet import MobileNet
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

import warnings
warnings.filterwarnings("ignore")

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
# Any results you write to the current directory are saved as output.

################                                           ####################
################             Reading the Data              ####################
################                                           ####################

imageSize=224

train_dir =  "G:/DL-Based Detection of Covid-19 Subjects using Radiographic Images/FULL FINAL DATASET/train/"
test_dir  =  "G:/DL-Based Detection of Covid-19 Subjects using Radiographic Images/FULL FINAL DATASET/test/"

from tqdm import tqdm

def get_data(folder):
    # Load the data and labels from the given folder.
    X = [] ; y = []
    for folderName in os.listdir(folder):
        if not folderName.startswith('.'):
            if (folderName in ['NORMAL']):
                label = 0
            elif folderName in ['BACTERIAL']:
                label = 1
            elif folderName in ['NON-COVID19 VIRAL']:
                label = 2
            elif folderName in ['COVID19 VIRAL']:
                label = 3
            else:
                label = 4
            for image_filename in tqdm(os.listdir(folder + folderName)):
                img_file = cv2.imread(folder + folderName + '/' + image_filename)
                if img_file is not None:
                    img_file = skimage.transform.resize(img_file, (imageSize, imageSize, 3))
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    y.append(label)
    X = np.asarray(X)
    y = np.asarray(y)
    return X,y

X_train, y_train = get_data(train_dir) # Un-comment to use full dataset: Step 1 of 2
X_test, y_test   = get_data(test_dir)

# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X_test, y_test, test_size=0.2, random_state=40) # Re-comment to use full dataset: Step 2 of 2

# Encode labels to hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])
from keras.utils.np_utils import to_categorical
y_trainHot = to_categorical(y_train, num_classes = 4)
y_testHot = to_categorical(y_test, num_classes = 4)

print(len(y_trainHot)) ; print(len(y_testHot))

def plotHistogram(a):
    # Plot histogram of RGB Pixel Intensities
    plt.figure(figsize=(10,5))
    plt.subplot(1,2,1)
    plt.imshow(a)
    plt.axis('off')
    histo = plt.subplot(1,2,2)
    histo.set_ylabel('Count')
    histo.set_xlabel('Pixel Intensity')
    n_bins = 30
    plt.hist(a[:,:,0].flatten(), bins= n_bins, lw = 0, color='r', alpha=0.5);
    plt.hist(a[:,:,1].flatten(), bins= n_bins, lw = 0, color='g', alpha=0.5);
    plt.hist(a[:,:,2].flatten(), bins= n_bins, lw = 0, color='b', alpha=0.5);
plotHistogram(X_train[1])

print("Normal")
multipleImages = glob("G:/DL-Based Detection of Covid-19 Subjects using Radiographic Images/FINAL AUGMENTED DATA/train/NORMAL/**")
i_ = 0
plt.rcParams['figure.figsize'] = (10.0, 10.0)
plt.subplots_adjust(wspace=0, hspace=0)
for l in multipleImages[:25]:
    im = cv2.imread(l)
    im = cv2.resize(im, (128, 128)) 
    plt.subplot(5, 5, i_+1) #.set_title(l)
    plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB)); plt.axis('off')
    i_ += 1
    
print("Bacterial Infection")
multipleImages = glob("G:/DL-Based Detection of Covid-19 Subjects using Radiographic Images/FINAL AUGMENTED DATA/train/BACTERIAL/**")
i_ = 0
plt.rcParams['figure.figsize'] = (10.0, 10.0)
plt.subplots_adjust(wspace=0, hspace=0)
for l in multipleImages[:25]:
    im = cv2.imread(l)
    im = cv2.resize(im, (128, 128)) 
    plt.subplot(5, 5, i_+1) #.set_title(l)
    plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB)); plt.axis('off')
    i_ += 1
        
print("COVID19 Viral Infection")
multipleImages = glob("G:/DL-Based Detection of Covid-19 Subjects using Radiographic Images/FINAL AUGMENTED DATA/train/COVID19 VIRAL/**")
i_ = 0
plt.rcParams['figure.figsize'] = (10.0, 10.0)
plt.subplots_adjust(wspace=0, hspace=0)
for l in multipleImages[:25]:
    im = cv2.imread(l)
    im = cv2.resize(im, (128, 128)) 
    plt.subplot(5, 5, i_+1) #.set_title(l)
    plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB)); plt.axis('off')
    i_ += 1
    
print("NON-COVID19 Viral Infection")
multipleImages = glob("G:/DL-Based Detection of Covid-19 Subjects using Radiographic Images/FINAL AUGMENTED DATA/train/NON-COVID19 VIRAL/**")
i_ = 0
plt.rcParams['figure.figsize'] = (10.0, 10.0)
plt.subplots_adjust(wspace=0, hspace=0)
for l in multipleImages[:25]:
    im = cv2.imread(l)
    im = cv2.resize(im, (128, 128)) 
    plt.subplot(5, 5, i_+1) #.set_title(l)
    plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB)); plt.axis('off')
    i_ += 1
    
map_characters = {0: 'Normal', 1: 'BACTERIAL', 2: 'NON-COVID19 VIRAL', 3: 'COVID19 VIRAL'}
dict_characters=map_characters
import seaborn as sns
df = pd.DataFrame()
df["labels"]=y_train
lab = df['labels']
dist = lab.value_counts()
sns.countplot(lab)
print(dict_characters)

# Helper Functions  Learning Curves and Confusion Matrix

from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """ This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True` """
    plt.figure(figsize = (5,5))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
map_characters1 = {0: 'Normal', 1: 'BACTERIAL', 2: 'NON-COVID19 VIRAL', 3: 'COVID19 VIRAL'}
class_weight1 = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train) ## ? 

weight_path1 = 'imagenet'
pretrained_model_1 = ResNet50(weights = weight_path1, include_top=False, input_shape=(imageSize, imageSize, 3))

for i in range(np.shape(X_train)[0]):
    for j in range(np.shape(X_train)[3]):        
        X_train[i,:,:,j] = X_train[i,:,:,j] - np.min(X_train[i,:,:,j])
        X_train[i,:,:,j] = X_train[i,:,:,j] / np.max(X_train[i,:,:,j])
        X_train[i,:,:,j] = 2* X_train[i,:,:,j] - 1

for i in range(np.shape(X_test)[0]):
    for j in range(np.shape(X_test)[3]):
        X_test[i,:,:,j] = (X_test[i,:,:,j] - np.min(X_test[i,:,:,j])) / np.max(X_test[i,:,:,j])
        X_test[i,:,:,j] = X_test[i,:,:,j] / np.max(X_test[i,:,:,j])
        X_test[i,:,:,j] = 2* X_test[i,:,:,j] - 1
        
# Deal with imbalanced class sizes below
# Make Data 1D for compatability upsampling methods
X_trainShape = X_train.shape[1]*X_train.shape[2]*X_train.shape[3]
X_testShape = X_test.shape[1]*X_test.shape[2]*X_test.shape[3]
X_trainFlat = X_train.reshape(X_train.shape[0], X_trainShape)
X_testFlat = X_test.reshape(X_test.shape[0], X_testShape)
Y_train = y_train
Y_test = y_test
#ros = RandomOverSampler(sampling_strategy='auto')
ros = RandomUnderSampler(sampling_strategy='auto')
X_trainRos, Y_trainRos = ros.fit_sample(X_trainFlat, Y_train)
#ros = ADASYN(sampling_strategy= 'not majority') 
ros = SMOTE(sampling_strategy='not majority', k_neighbors= 7)
#ros = SVMSMOTE(sampling_strategy='not majority')
X_testRos, Y_testRos = ros.fit_sample(X_testFlat, Y_test)
# Encode labels to hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])
Y_trainRosHot = to_categorical(Y_trainRos, num_classes = 4)
Y_testRosHot = to_categorical(Y_testRos, num_classes = 4)
# Make Data 2D again
for i in range(len(X_trainRos)):
    height, width, channels = imageSize,imageSize,3
    X_trainRosReshaped = X_trainRos.reshape(len(X_trainRos),height,width,channels)
for i in range(len(X_testRos)):
    height, width, channels = imageSize,imageSize,3
    X_testRosReshaped = X_testRos.reshape(len(X_testRos),height,width,channels)
# Plot Label Distribution
dfRos = pd.DataFrame()
dfRos["labels"]=Y_trainRos
labRos = dfRos['labels']
distRos = lab.value_counts()
sns.countplot(labRos)
print(dict_characters)

class_weight1 = class_weight.compute_class_weight('balanced', np.unique(Y_train), Y_train)
print("Old Class Weights: ",class_weight1)
class_weight2 = class_weight.compute_class_weight('balanced', np.unique(Y_trainRos), Y_trainRos)
print("New Class Weights: ",class_weight2)

# Shuffle data to permit further subsampling
from sklearn.utils import shuffle
X_trainRosReshaped, Y_trainRosHot = shuffle(X_trainRosReshaped, Y_trainRosHot, random_state=13)
X_testRosReshaped, Y_testRosHot = shuffle(X_testRosReshaped, Y_testRosHot, random_state=13)

def get_model(pretrainedmodel,numclasses):
#   base_model = pretrained_model_1 # Topless
    base_model = pretrainedmodel # Topless
    # Add top layer
    x = base_model.output
#   x = Conv2D(512, kernel_size=(3,3), padding = 'valid')(x)
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
#   x = Flatten()(x)
#   x = Dropout(0.2)(x)
    predictions = Dense(numclasses, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    # Train top layer
    for layer in base_model.layers:
        layer.trainable = True
# =============================================================================
#     for layer in base_model.layers:
#         if isinstance(layer, BatchNormalization):
#             layer.trainable = True
#         else:
#             layer.trainable = False
# =============================================================================
    return model

model = get_model(pretrained_model_1, 4)
model.summary()

optimizer1 = keras.optimizers.Adam(lr=1e-5)
optimizer2 = keras.optimizers.RMSprop(lr=0.0001/2)
optimizer3 = keras.optimizers.SGD(lr=0.0001)

model.compile(loss='categorical_crossentropy', 
              optimizer=optimizer1, 
              metrics=['accuracy'])

callbacks_list = EarlyStopping(monitor='val_loss', patience=6, verbose=1)
best_model     = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

import tensorflow as tf
from tensorflow.compat.v1.keras.backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.compat.v1.Session(config=config)
set_session(sess)

# Fit model
xtrain = X_trainRosReshaped
ytrain = Y_trainRosHot
xtest = X_testRosReshaped
ytest = Y_testRosHot
numepochs = 50
classweight = class_weight2 
batch_size = 16

###############################################################################
##########                  Data Augmentation                        ##########
###############################################################################

# =============================================================================
# from keras.preprocessing.image import ImageDataGenerator
# 
# data_aug = ImageDataGenerator(rotation_range=5, 
#                               width_shift_range=0.1, 
#                               height_shift_range=0.1, 
#                               brightness_range=[0.8,1.2], 
#                               fill_mode='nearest')
# 
# i = 0
# 
# while i < 1:
#     
#     for X_batch in data_aug.flow_from_directory(
#             directory = 'G:/DL-Based Detection of Covid-19 Subjects using Radiographic Images/FULL FINAL DATASET/train/',
#             classes   = ['NON-COVID19 VIRAL'],
#             target_size=(496,768),
#             batch_size= len(X_train),
#             color_mode='rgb',
#             class_mode=None,
#             save_format='jpeg',
#             save_to_dir='G:/DL-Based Detection of Covid-19 Subjects using Radiographic Images/FINAL AUGMENTED DATA/train/NON-COVID19 VIRAL/',
#             save_prefix='aug'):
#         
#         break
#     
#     i += 1
# =============================================================================

###############################################################################
##########                    Model Fitting                          ##########
###############################################################################

history = model.fit(xtrain,ytrain, epochs=numepochs, batch_size = batch_size, class_weight=classweight, validation_split = 0.2, verbose=1, callbacks = [callbacks_list, best_model])
labels = map_characters1

# Evaluate model
score = model.evaluate(xtest,ytest, verbose=1)
print('\nKeras CNN - accuracy:', score[1], '\n')
y_pred = model.predict(xtest)
print('\n', sklearn.metrics.classification_report(np.where(ytest > 0)[1], np.argmax(y_pred, axis=1), target_names=list(labels.values())), sep='') 
Y_pred_classes = np.argmax(y_pred,axis = 1) 
Y_true = np.argmax(ytest,axis = 1)
y_true = Y_true
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 

###############################################################################
##########                           Plots                           ##########
###############################################################################

# Plot training & validation accuracy values
plt.figure(figsize=(10,4))
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
# plt.savefig('G:/Prof. Soltanian-Zadeh, CT DATA/Accuracy_ResNet50.jpg', quality= 70, dpi= 500)
plt.show()

# Plot training & validation loss values
plt.figure(figsize=(10,4))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
#plt.savefig('G:/Prof. Soltanian-Zadeh, CT DATA/Loss_ResNet50.jpg', quality= 70, dpi= 500)
plt.show()

plot_confusion_matrix(confusion_mtx, classes = list(labels.values()))
#plt.savefig('G:/Prof. Soltanian-Zadeh, CT DATA/ConfMat_ResNet50.jpg', quality= 70, dpi= 500)

###############################################################################
##########               Class Activation Maps (CAMs)                ##########
###############################################################################

import matplotlib.image as mpimg
from keras import backend as K
from keras.preprocessing import image
import numpy as np
import cv2

img_path = ...

preds     = np.zeros([50,4])
preds_max = np.zeros(50)

for count, img_filename in enumerate(os.listdir(img_path)):
    img = image.load_img(img_path + '/' + img_filename, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x - np.min(x)
    x /= np.max(x)
    x = x*2 - 1
    preds[count,:]   = model.predict(x)[0]
    preds_max[count] = np.argmax(preds) % 4
    output = model.output[:, np.int(preds_max[count])]
    last_conv_layer = model.get_layer('res5c_branch2c')
    grads = K.gradients(output, last_conv_layer.output)[0]
    pooled_grads = K.mean(grads, axis=(0, 1, 2))
    iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])
    pooled_grads_value, conv_layer_output_value = iterate([x])
    for i in range(512):
      conv_layer_output_value[:, :, i] *= pooled_grads_value[i]
    heatmap = np.mean(conv_layer_output_value, axis=-1)
    heatmap -= np.mean(heatmap)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)
    # heatmap = np.maximum(heatmap, 0)
    # heatmap /= np.max(heatmap)

    # plt.matshow(heatmap)
    # plt.show()
    
    img = cv2.imread(img_path + '/' + img_filename)
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    hif = .6
    superimposed_img = heatmap * hif + img
    output = ...
    cv2.imwrite(output %(count), superimposed_img)
    # img=mpimg.imread(...)
    # plt.imshow(img)
    print(count)